In [4]:
import numpy as np
import pandas as pd
from openai import OpenAI
import yaml
import json
import os
from dotenv import load_dotenv

In [5]:
from sentence_transformers import SentenceTransformer, util
from functools import lru_cache

@lru_cache(maxsize=None)
def load_model():
    return SentenceTransformer('all-MiniLM-L6-v2')

model = SentenceTransformer('all-MiniLM-L6-v2')

.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [6]:
# Load the Excel file
excel_file = "data/Data_analyst_question.xlsx"
df = pd.read_excel(excel_file)

In [7]:
#selected_questions
df

,Category,Question,Correct Answer
0,R,How can you load a .csv file in R?,Loading a .csv file in R is quite easy. All yo...
1,R,What is Rmarkdown? What is the use of it?,RMarkdown is a reporting tool provided by R. W...
2,R,How do you install a package in R?,To install a package in R: install.packages(“<...
3,R,What are the steps to build and evaluate a lin...,These are sequential steps which need to be fo...
4,R,"Name some packages in R, which can be used for...",These are some packages in R which can used fo...
...,...,...,...
101,Data Structure,What is an asymptotic analysis of an algorithm?,Asymptotic analysis of an algorithm defines th...
102,Data Structure,What is hashmap in data structure?,Hashmap is a data structure that uses an imple...
103,Data Structure,What is the requirement for an object to be us...,The key or value object that gets used in the ...
104,Data Structure,What is the time complexity of basic operation...,The time complexity is O(1) assuming that the ...


In [8]:
# Ask the user to choose a category from column A (Category)
print("Available Categories:")
categories = df["Category"].unique()
for idx, category in enumerate(categories):
    print(f"{idx + 1}. {category}")

category_choice = int(input("Choose a category (enter the corresponding number): ")) - 1
selected_category = categories[category_choice]

Available Categories:
1. R
2. Python
3. Data Warehouse
4. Database
5. MySQL
6. Algorithms
7. Data Structure


In [9]:
category_data = df[df["Category"] == selected_category]
category_data = category_data.sample(frac=1).reset_index(drop=True)

In [10]:
category_data

,Category,Question,Correct Answer
0,Python,What is PIP?,PIP denotes Python Installer Package. It is us...
1,Python,How Can You Copy Objects in Python?,The function used to copy objects in Python ar...
2,Python,How Do You Compute the Euclidean Distance Betw...,">> #Input\n\n>>p = pd.Series([1, 2, 3, 4, 5, 6..."
3,Python,How to combine dataframes in Pandas?,The following are the ways through which the d...
4,Python,What are Python libraries?,A Python library is a group of Python packages...
5,Python,What do you mean by underfitting a dataset?,Underfitting a dataset means our model is fitt...
6,Python,What Is the Difference Between Matrices and Ar...,A matrix comes from linear algebra and is a tw...
7,Python,What is the Lambda function and why it is used...,A lambda function is a type of anonymous funct...
8,Python,Write a program that checks if all of the numb...,def check_distinct(data_list):\n\n if len(data...
9,Python,What Advantage Does the Numpy Array Have over ...,Numpy is written in C so that all its complexi...


In [ ]:
user_answers = []
questions = []
correct_answers = []

num_questions = min(2, len(category_data))
feedback = ""

for i in range(num_questions):
    question = category_data.loc[i, "Question"]
    questions.append(question)

    correct_answer = category_data.loc[i, "Correct Answer"]
    correct_answers.append(correct_answer)

    user_answer = input(f"Q: {question}\nA: ")
    user_answers.append(user_answer)


Q: What are built-in types of Python?
A: string
Q: How Do You Compute the Euclidean Distance Between Two Series, from 1 to 10?
A: multiply


In [ ]:
scores = []
for u, q, c in zip(user_answers, questions, correct_answers):
  user_ans_doc = nlp(u)
  correct_ans_doc = nlp(c)
  similarity_score = user_ans_doc.similarity(correct_ans_doc)
  scores.append(similarity_score)

<ipython-input-27-d756c9c5791b>:9: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  similarity_score = user_ans_doc.similarity(correct_ans_doc)


In [ ]:
# Provide personalized feedback to the user based on semantic analysis
#feedback += f"Question: {question}\n"
#feedback += f"Your Answer: {user_answer}\n"
#feedback += f"Correct Answer: {correct_answer}\n"
#feedback += f"Your Score: {similarity_score:.2f}\n"
#feedback += "\n"
for u, q, c, s in zip(user_answers, questions, correct_answers, scores):
  feedback += f"Question: {q}\n"
  feedback += f"Your Answer: {u}\n"
  feedback += f"Correct Answer: {c}\n"
  feedback += f"Your Score: {s:.2f}\n"
  feedback += "\n"
# Display personalized feedback to the user
print("\nFeedback:")
print(feedback)


Feedback:
Question: What are built-in types of Python?
Your Answer: string
Correct Answer: Built in functions, Boolean, String, Complex numbers, Floating point, Integers
Your Score: 0.24

Question: How Do You Compute the Euclidean Distance Between Two Series, from 1 to 10?
Your Answer: multiply
Correct Answer: >> #Input

>>p = pd.Series([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

>>q = pd.Series([10, 9, 8, 7, 6, 5, 4, 3, 2, 1])

>> #Solution

>>sum((p - q)**2)**.5

>> #Solution using func

>>np.linalg.norm(p-q)
Your Score: -0.04


